# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f3590472ee0>
├── 'a' --> tensor([[-1.5548,  0.6711,  0.0644],
│                   [-0.7434, -0.3867,  0.7153]])
└── 'x' --> <FastTreeValue 0x7f359046d5e0>
    └── 'c' --> tensor([[-1.4016, -1.3053, -1.2831, -1.4641],
                        [-0.9743,  0.9682,  0.3768, -1.1201],
                        [-0.6099, -1.0846, -0.3544,  1.9093]])

In [4]:
t.a

tensor([[-1.5548,  0.6711,  0.0644],
        [-0.7434, -0.3867,  0.7153]])

In [5]:
%timeit t.a

63.9 ns ± 0.125 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f3590472ee0>
├── 'a' --> tensor([[-1.5840, -0.4415,  0.9279],
│                   [ 0.5243, -1.6960, -0.0154]])
└── 'x' --> <FastTreeValue 0x7f359046d5e0>
    └── 'c' --> tensor([[-1.4016, -1.3053, -1.2831, -1.4641],
                        [-0.9743,  0.9682,  0.3768, -1.1201],
                        [-0.6099, -1.0846, -0.3544,  1.9093]])

In [7]:
%timeit t.a = new_value

71 ns ± 0.0709 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-1.5548,  0.6711,  0.0644],
               [-0.7434, -0.3867,  0.7153]]),
    x: Batch(
           c: tensor([[-1.4016, -1.3053, -1.2831, -1.4641],
                      [-0.9743,  0.9682,  0.3768, -1.1201],
                      [-0.6099, -1.0846, -0.3544,  1.9093]]),
       ),
)

In [10]:
b.a

tensor([[-1.5548,  0.6711,  0.0644],
        [-0.7434, -0.3867,  0.7153]])

In [11]:
%timeit b.a

57.7 ns ± 0.214 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.2956, -0.1550, -1.4741],
               [ 1.2510,  0.7931,  0.2571]]),
    x: Batch(
           c: tensor([[-1.4016, -1.3053, -1.2831, -1.4641],
                      [-0.9743,  0.9682,  0.3768, -1.1201],
                      [-0.6099, -1.0846, -0.3544,  1.9093]]),
       ),
)

In [13]:
%timeit b.a = new_value

493 ns ± 0.273 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

851 ns ± 1.05 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.6 µs ± 28.5 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

144 µs ± 382 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

146 µs ± 5.37 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f34b9246520>
├── 'a' --> tensor([[[-1.5548,  0.6711,  0.0644],
│                    [-0.7434, -0.3867,  0.7153]],
│           
│                   [[-1.5548,  0.6711,  0.0644],
│                    [-0.7434, -0.3867,  0.7153]],
│           
│                   [[-1.5548,  0.6711,  0.0644],
│                    [-0.7434, -0.3867,  0.7153]],
│           
│                   [[-1.5548,  0.6711,  0.0644],
│                    [-0.7434, -0.3867,  0.7153]],
│           
│                   [[-1.5548,  0.6711,  0.0644],
│                    [-0.7434, -0.3867,  0.7153]],
│           
│                   [[-1.5548,  0.6711,  0.0644],
│                    [-0.7434, -0.3867,  0.7153]],
│           
│                   [[-1.5548,  0.6711,  0.0644],
│                    [-0.7434, -0.3867,  0.7153]],
│           
│                   [[-1.5548,  0.6711,  0.0644],
│                    [-0.7434, -0.3867,  0.7153]]])
└── 'x' --> <FastTreeValue 0x7f34b9246040>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.7 µs ± 56.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f3593501700>
├── 'a' --> tensor([[-1.5548,  0.6711,  0.0644],
│                   [-0.7434, -0.3867,  0.7153],
│                   [-1.5548,  0.6711,  0.0644],
│                   [-0.7434, -0.3867,  0.7153],
│                   [-1.5548,  0.6711,  0.0644],
│                   [-0.7434, -0.3867,  0.7153],
│                   [-1.5548,  0.6711,  0.0644],
│                   [-0.7434, -0.3867,  0.7153],
│                   [-1.5548,  0.6711,  0.0644],
│                   [-0.7434, -0.3867,  0.7153],
│                   [-1.5548,  0.6711,  0.0644],
│                   [-0.7434, -0.3867,  0.7153],
│                   [-1.5548,  0.6711,  0.0644],
│                   [-0.7434, -0.3867,  0.7153],
│                   [-1.5548,  0.6711,  0.0644],
│                   [-0.7434, -0.3867,  0.7153]])
└── 'x' --> <FastTreeValue 0x7f34cb83ca00>
    └── 'c' --> tensor([[-1.4016, -1.3053, -1.2831, -1.4641],
                        [-0.9743,  0.9682,  0.3768, -1.1201],
                 

In [23]:
%timeit t_cat(trees)

30.7 µs ± 26.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

66.5 µs ± 50.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-1.5548,  0.6711,  0.0644],
                [-0.7434, -0.3867,  0.7153]],
       
               [[-1.5548,  0.6711,  0.0644],
                [-0.7434, -0.3867,  0.7153]],
       
               [[-1.5548,  0.6711,  0.0644],
                [-0.7434, -0.3867,  0.7153]],
       
               [[-1.5548,  0.6711,  0.0644],
                [-0.7434, -0.3867,  0.7153]],
       
               [[-1.5548,  0.6711,  0.0644],
                [-0.7434, -0.3867,  0.7153]],
       
               [[-1.5548,  0.6711,  0.0644],
                [-0.7434, -0.3867,  0.7153]],
       
               [[-1.5548,  0.6711,  0.0644],
                [-0.7434, -0.3867,  0.7153]],
       
               [[-1.5548,  0.6711,  0.0644],
                [-0.7434, -0.3867,  0.7153]]]),
    x: Batch(
           c: tensor([[[-1.4016, -1.3053, -1.2831, -1.4641],
                       [-0.9743,  0.9682,  0.3768, -1.1201],
                       [-0.6099, -1.0846, -0.3544,  1.9093]],
         

In [26]:
%timeit Batch.stack(batches)

79.3 µs ± 284 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-1.5548,  0.6711,  0.0644],
               [-0.7434, -0.3867,  0.7153],
               [-1.5548,  0.6711,  0.0644],
               [-0.7434, -0.3867,  0.7153],
               [-1.5548,  0.6711,  0.0644],
               [-0.7434, -0.3867,  0.7153],
               [-1.5548,  0.6711,  0.0644],
               [-0.7434, -0.3867,  0.7153],
               [-1.5548,  0.6711,  0.0644],
               [-0.7434, -0.3867,  0.7153],
               [-1.5548,  0.6711,  0.0644],
               [-0.7434, -0.3867,  0.7153],
               [-1.5548,  0.6711,  0.0644],
               [-0.7434, -0.3867,  0.7153],
               [-1.5548,  0.6711,  0.0644],
               [-0.7434, -0.3867,  0.7153]]),
    x: Batch(
           c: tensor([[-1.4016, -1.3053, -1.2831, -1.4641],
                      [-0.9743,  0.9682,  0.3768, -1.1201],
                      [-0.6099, -1.0846, -0.3544,  1.9093],
                      [-1.4016, -1.3053, -1.2831, -1.4641],
                      [-0.9743,  

In [28]:
%timeit Batch.cat(batches)

149 µs ± 165 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

342 µs ± 2.89 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
